In [1]:
import pandas as pd
import os


relative_data_path = '../../data/raw/' 
file_name = 'X01_5대범죄발생현황.csv'
file_path = os.path.join(relative_data_path, file_name)

# 2. CSV 파일 읽기
try:
    df_crime = pd.read_csv(file_path, encoding='utf-8', header=[0,1,2,3])
    
    # 데이터가 잘 불러와졌는지 확인하기 위해 상위 5개 행을 출력
    print("데이터 불러오기 성공!")
    print(df_crime.head())
    
except FileNotFoundError:
    print(f"오류: '{file_path}' 파일을 찾을 수 없습니다. 파일 경로와 이름을 다시 확인해주세요.")
except Exception as e:
    # 한글 인코딩 문제로 오류가 날 경우를 대비한 추가 처리
    print(f"다른 인코딩으로 재시도 중... 현재 오류: {e}")
    try:
        df_crime = pd.read_csv(file_path, encoding='cp949') # 윈도우 기본 인코딩
        print("데이터 불러오기 성공 (cp949)")
        print(df_crime.head())
    except:
        print("파일 불러오기에 실패했습니다. 파일이 깨졌거나 인코딩 문제가 심각합니다.")

데이터 불러오기 성공!
  자치구별(1) 자치구별(2)    2014                                           ... 2024  \
  자치구별(1) 자치구별(2)      합계                                           ...   합계   
  자치구별(1) 자치구별(2)      소계          살인        강도      강간·강제추행        ...   살인   
  자치구별(1) 자치구별(2)      발생     검거   발생   검거   발생   검거      발생    검거  ...   발생   
0      합계      소계  130674  79061  158  152  343  299    5462  4957  ...  149   
1      합계     종로구    5021   4610    3    7   12   15     226   948  ...    2   
2      합계      중구    5231   3188    6    5   13   13     221   161  ...    5   
3      합계     용산구    3799   2340    1    2    7    7     213   169  ...   10   
4      합계     성동구    3582   2048    1    1    5    6     141   103  ...    5   

                                                          
                                                          
        강도     강간·강제추행           절도            폭력         
    검거  발생  검거      발생    검거     발생     검거     발생     검거  
0  137  87  85    5523  5183  

In [2]:
df_crime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Columns: 134 entries, ('자치구별(1)', '자치구별(1)', '자치구별(1)', '자치구별(1)') to ('2024', '합계', '폭력', '검거')
dtypes: int64(124), object(10)
memory usage: 27.3+ KB


In [3]:
df_crime.columns

MultiIndex([('자치구별(1)', '자치구별(1)', '자치구별(1)', '자치구별(1)'),
            ('자치구별(2)', '자치구별(2)', '자치구별(2)', '자치구별(2)'),
            (   '2014',      '합계',      '소계',      '발생'),
            (   '2014',      '합계',      '소계',      '검거'),
            (   '2014',      '합계',      '살인',      '발생'),
            (   '2014',      '합계',      '살인',      '검거'),
            (   '2014',      '합계',      '강도',      '발생'),
            (   '2014',      '합계',      '강도',      '검거'),
            (   '2014',      '합계', '강간·강제추행',      '발생'),
            (   '2014',      '합계', '강간·강제추행',      '검거'),
            ...
            (   '2024',      '합계',      '살인',      '발생'),
            (   '2024',      '합계',      '살인',      '검거'),
            (   '2024',      '합계',      '강도',      '발생'),
            (   '2024',      '합계',      '강도',      '검거'),
            (   '2024',      '합계', '강간·강제추행',      '발생'),
            (   '2024',      '합계', '강간·강제추행',      '검거'),
            (   '2024',      '합계',      '절도',      '발생')

In [4]:
df_crime.head()

자치구별(1) 자치구별(2)    2014                                           ... 2024  \
  자치구별(1) 자치구별(2)      합계                                           ...   합계   
  자치구별(1) 자치구별(2)      소계          살인        강도      강간·강제추행        ...   살인   
  자치구별(1) 자치구별(2)      발생     검거   발생   검거   발생   검거      발생    검거  ...   발생   
0      합계      소계  130674  79061  158  152  343  299    5462  4957  ...  149   
1      합계     종로구    5021   4610    3    7   12   15     226   948  ...    2   
2      합계      중구    5231   3188    6    5   13   13     221   161  ...    5   
3      합계     용산구    3799   2340    1    2    7    7     213   169  ...   10   
4      합계     성동구    3582   2048    1    1    5    6     141   103  ...    5   

                                                          
                                                          
        강도     강간·강제추행           절도            폭력         
    검거  발생  검거      발생    검거     발생     검거     발생     검거  
0  137  87  85    5523  5183  35506  22796  39554  35074  
1    3   5  10     209   933   1183   1056   1366   1246  
2    4   5   5     177   121   1398    816   1370   1234  
3    6   4   3     267   235   1082    589   1959   1642  
4    6   3   3     114    90    966    544   1029    867  

[5 rows x 134 columns]

1. 발생과 검거가 나눠져 있는 점 > 검거율 컬럼 생성하기 
2. 년/월로 나누어져 있는 점 

In [5]:
# 1. 컬럼 이름 재정의 (월 정보 제거 및 필수 정보만 남기기)
multi_cols = df_crime.columns.tolist()
new_cols = []
for col_tuple in multi_cols:
    # 0번 레벨(2014.1)에서 '.' 앞부분만 추출하여 '년도'로 사용
    year_part = str(col_tuple[0]).split('.')[0] 
    
    # 2번 레벨(범죄 유형)과 3번 레벨(발생/검거) 사용
    crime_type = col_tuple[2]
    classification = col_tuple[3]
    
    new_cols.append((year_part, crime_type, classification))

# 새로운 컬럼 이름을 데이터프레임에 적용
df_crime.columns = pd.MultiIndex.from_tuples(new_cols)

df_crime.head()

자치구별(1) 자치구별(2)    2014                                           ... 2024  \
  자치구별(1) 자치구별(2)      소계          살인        강도      강간·강제추행        ...   살인   
  자치구별(1) 자치구별(2)      발생     검거   발생   검거   발생   검거      발생    검거  ...   발생   
0      합계      소계  130674  79061  158  152  343  299    5462  4957  ...  149   
1      합계     종로구    5021   4610    3    7   12   15     226   948  ...    2   
2      합계      중구    5231   3188    6    5   13   13     221   161  ...    5   
3      합계     용산구    3799   2340    1    2    7    7     213   169  ...   10   
4      합계     성동구    3582   2048    1    1    5    6     141   103  ...    5   

                                                          
        강도     강간·강제추행           절도            폭력         
    검거  발생  검거      발생    검거     발생     검거     발생     검거  
0  137  87  85    5523  5183  35506  22796  39554  35074  
1    3   5  10     209   933   1183   1056   1366   1246  
2    4   5   5     177   121   1398    816   1370   1234  
3    6   4   3     267   235   1082    589   1959   1642  
4    6   3   3     114    90    966    544   1029    867  

[5 rows x 134 columns]

In [6]:
# 1. 자치구 이름이 있는 컬럼(인덱스 1)을 인덱스로 설정
# 컬럼 위치 1에 해당하는 컬럼 이름(튜플)을 찾아서 인덱스로 설정
name = df_crime.columns[1] 
df_final= df_crime.set_index(name)

# 3. 불필요한 첫 번째 정보성 컬럼 드롭 (Level 0의 첫 번째 컬럼)
df_final = df_final.drop(columns=df_final.columns[0], level=0, errors='ignore')


In [7]:
# 데이터 컬럼만 선택 (자치구 인덱스를 제외한 모든 컬럼)
data_cols = df_final.iloc[:, :]

# apply(pd.to_numeric, errors='coerce')를 사용해 숫자 변환
# errors='coerce'는 숫자로 변환 불가능한 값은 NaN으로 만든다.
df_final.loc[:, data_cols.columns] = data_cols.apply(pd.to_numeric, errors='coerce')

# NaN(결측치)을 0으로 채우기 (범죄 건수이므로)
df_final = df_final.fillna(0)

print("✅ 3단계: 데이터 숫자형으로 변환 및 중복 연도 정리 완료!")

✅ 3단계: 데이터 숫자형으로 변환 및 중복 연도 정리 완료!


C:\Users\user\AppData\Local\Temp\ipykernel_1488\2740209701.py:9: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_final = df_final.fillna(0)


In [8]:
new_df_crime=df_final.iloc[1:,1:]

new_df_crime.head()

2014                                         \
                               소계       살인     강도     강간·강제추행         절도   
                               발생    검거 발생 검거  발생  검거      발생   검거    발생   
(자치구별(2), 자치구별(2), 자치구별(2))                                                
종로구                          5021  4610  3  7  12  15     226  948  2272   
중구                           5231  3188  6  5  13  13     221  161  2576   
용산구                          3799  2340  1  2   7   7     213  169  1560   
성동구                          3582  2048  1  1   5   6     141  103  1753   
광진구                          6268  3531  8  9  16  22     249  199  3167   

                                   ... 2024                                   \
                                   ...   살인      강도       강간·강제추행         절도   
                               검거  ...   발생 검거   발생    검거      발생   검거    발생   
(자치구별(2), 자치구별(2), 자치구별(2))        ...                                         
종로구                          1281  ...    2  3  5.0  10.0     209  933  1183   
중구                            887  ...    5  4  5.0   5.0     177  121  1398   
용산구                           522  ...   10  6  4.0   3.0     267  235  1082   
성동구                           529  ...    5  6  3.0   3.0     114   90   966   
광진구                          1001  ...    2  2  6.0   7.0     235  183  1320   

                                               
                                     폭력        
                               검거    발생    검거  
(자치구별(2), 자치구별(2), 자치구별(2))                    
종로구                          1056  1366  1246  
중구                            816  1370  1234  
용산구                           589  1959  1642  
성동구                           544  1029   867  
광진구                           942  1307  1085  

[5 rows x 132 columns]

1. 년도별 인구당 여성 치안 취약 범죄 발생률
2. 장기 누적 통계분석 (총 누적 발생 건수 / 장기 누적 검거율)
> 구조적인 치안 취약성을 증명

In [9]:
# ===========================================
# ✅ 중복 없는 연도 합계 생성 함수
#    - 연도 필터
#    - '총계' 유형(계/소계/합계/총계/5대범죄)만 사용(있을 때)
#    - (연도, 발생/검거)로 축소 후 연도 across 합산
# ===========================================
total_alias = ['계', '소계', '합계', '총계', '5대범죄']

def make_summary(df_src, year_start, year_end):
    years = [str(y) for y in range(year_start, year_end+1)]
    # 1) 해당 연도만 선택
    mask_year = df_src.columns.get_level_values(0).isin(years)
    df_y = df_src.loc[:, mask_year]

    # 2) '총계' 유형이 있으면 그것만, 없으면 전체 유형 사용
    has_total = df_y.columns.get_level_values(1).isin(total_alias).any()
    if has_total:
        df_y = df_y.loc[:, df_y.columns.get_level_values(1).isin(total_alias)]

    # 3) (연도, 발생/검거) 기준으로 유형(level=1)을 접고 합산
    #    -> 월(이미 제거됨)과 세부유형 중복 제거
    df_compact = df_y.groupby(level=[0, 2], axis=1).sum()

    # 4) 발생/검거 연도합 (컬럼: (연도, '발생'/'검거') 형태)
    발생_연도별 = df_compact.xs('발생', axis=1, level=1)
    검거_연도별 = df_compact.xs('검거', axis=1, level=1)

    # 연도 across 합산 (2014~2024 전체 누적 등)
    발생합 = 발생_연도별.sum(axis=1)
    검거합 = 검거_연도별.sum(axis=1)

    검거율 = (검거합 / 발생합 * 100).round(2)

    out = (
        pd.DataFrame({
            '지역구': df_compact.index,
            '총_누적_발생_건수': 발생합.astype(int),
            '총_누적_검거_건수': 검거합.astype(int),
            '장기_누적_검거율(%)': 검거율
        })
        .sort_values('지역구')
        .reset_index(drop=True)
    )
    return out

# ===========================================
# ✅ 위 함수로 2014~2024 / 2020~2024 요약 생성
#    (df_final은 네 전처리 단계에서 만든, 컬럼이 (연도, 범죄유형, 발생/검거) 인 DF)
# ===========================================
summary_2014_2024 = make_summary(df_final, 2014, 2024)
summary_2020_2024 = make_summary(df_final, 2020, 2024)
summary_2023 = make_summary(df_final, 2023, 2023)


# ===========================================
# ✅ 저장
# ===========================================
save_dir = '../../data/preprocessing'
os.makedirs(save_dir, exist_ok=True)

file_2014_2024 = os.path.join(save_dir, 'X01_전처리_5대범죄_발생검거_누적건수_2014_2024.csv')
file_2020_2024 = os.path.join(save_dir, 'X01_전처리_5대범죄_발생검거_누적건수_2020_2024.csv')
file_2023 = os.path.join(save_dir, 'X01_전처리_5대범죄_발생검거_누적건수_2023.csv')

summary_2014_2024.to_csv(file_2014_2024, encoding='utf-8-sig', index=False)
summary_2020_2024.to_csv(file_2020_2024, encoding='utf-8-sig', index=False)
summary_2023.to_csv(file_2023, encoding='utf-8-sig', index=False)

print("✅ 저장 완료")
print(f"전체 기간 파일: {file_2014_2024} | shape: {summary_2014_2024.shape}")
print(f"2020~2024 파일: {file_2020_2024} | shape: {summary_2020_2024.shape}")
print(f"2023년 파일: {file_2023} | shape: {summary_2023.shape}")
print(summary_2014_2024.head())

✅ 저장 완료
전체 기간 파일: ../../data/preprocessing\X01_전처리_5대범죄_발생검거_누적건수_2014_2024.csv | shape: (26, 4)
2020~2024 파일: ../../data/preprocessing\X01_전처리_5대범죄_발생검거_누적건수_2020_2024.csv | shape: (26, 4)
2023년 파일: ../../data/preprocessing\X01_전처리_5대범죄_발생검거_누적건수_2023.csv | shape: (26, 4)
   지역구  총_누적_발생_건수  총_누적_검거_건수  장기_누적_검거율(%)
0  강남구       81473       56709         69.60
1  강동구       44997       31491         69.98
2  강북구       35873       28549         79.58
3  강서구       52290       39434         75.41
4  관악구       58521       40370         68.98


C:\Users\user\AppData\Local\Temp\ipykernel_1488\2941124700.py:22: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df_compact = df_y.groupby(level=[0, 2], axis=1).sum()
C:\Users\user\AppData\Local\Temp\ipykernel_1488\2941124700.py:22: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df_compact = df_y.groupby(level=[0, 2], axis=1).sum()
C:\Users\user\AppData\Local\Temp\ipykernel_1488\2941124700.py:22: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df_compact = df_y.groupby(level=[0, 2], axis=1).sum()
